# Saving field data to hdf5 files

In [21]:
import dolfin as df
from finmag import Simulation as Sim
from finmag.energies import Exchange, Zeeman
from finmag.util.consts import mu0

First, we create the simulation object and initialise the magnetisation:

In [22]:
mesh = df.RectangleMesh(df.Point(0, 0), df.Point(100, 100), 10, 10)

Ms = 1e6
A = 1e-11

sim = Sim(mesh, Ms, unit_length=1e-9)

sim.add(Exchange(A))
sim.add(Zeeman((0, 0, 0.1/mu0)))
sim.set_m((1, 0, 0))

[2015-10-02 13:23:10] INFO: Finmag logging output will be written to file: '/home/mb4e10/finmag/doc/ipython_notebooks_src/unnamed.log' (any old content will be overwritten).
[2015-10-02 13:23:10] DEBUG: Creating DataWriter for file 'unnamed.ndt'
[2015-10-02 13:23:10] INFO: Creating Sim object name='unnamed', instance_id=8 (rank=0/1).
[2015-10-02 13:23:10] DEBUG:    Total number of Sim objects in this session: 9
[2015-10-02 13:23:10] INFO: <Mesh of topological dimension 2 (triangles) with 121 vertices and 200 cells, ordered>
[2015-10-02 13:23:10] DEBUG: Creating LLG object.
[2015-10-02 13:23:10] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2015-10-02 13:23:10] DEBUG: Adding interaction Exchange to simulation.
[2015-10-02 13:23:10] DEBUG: Adding interaction Zeeman to simulation.


## Saving field

Now, we run the simulation in uniform timesteps ($\Delta t = 20 \,\text{ps}$) for 0.5 ns and in every time step, we save the magnetisation field.

In every step, we provide both the name and time at which the magnetisation is saved.

In [23]:
import numpy as np

filename = 'magnetisation'

T = 0.01e-9
dt = 2e-12

t_array = np.arange(0, T+dt/2, dt)
for t in t_array:
    sim.run_until(t)
    sim.llg.m_field.save_hdf5(filename, t=sim.t)

[2015-10-02 13:23:12] INFO: Simulation will run until t = 0 s.
[2015-10-02 13:23:12] INFO: Creating integrator with backend sundials and arguments {'reltol': 1e-06, 'abstol': 1e-06}.
[2015-10-02 13:23:12] DEBUG: Updating get method for steps in TableWriter(name=unnamed.ndt)
[2015-10-02 13:23:12] DEBUG: Updating get method for last_step_dt in TableWriter(name=unnamed.ndt)
[2015-10-02 13:23:12] DEBUG: Updating get method for dmdt in TableWriter(name=unnamed.ndt)
[2015-10-02 13:23:12] INFO: Simulation has reached time t = 0 s.
[2015-10-02 13:23:12] INFO: Simulation will run until t = 2e-12 s.
[2015-10-02 13:23:12] INFO: Simulation has reached time t = 2e-12 s.
[2015-10-02 13:23:12] INFO: Simulation will run until t = 4e-12 s.
[2015-10-02 13:23:12] INFO: Simulation has reached time t = 4e-12 s.
[2015-10-02 13:23:12] INFO: Simulation will run until t = 6e-12 s.
[2015-10-02 13:23:12] INFO: Simulation has reached time t = 6e-12 s.
[2015-10-02 13:23:12] INFO: Simulation will run until t = 8e-1

Debug: (0/1) opening file magnetisation.h5
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json


We can now see that both h5 and json files are created.

In [25]:
!ls *.json
!ls *.h5

magnetisation.json
magnetisation.h5


## Loading field

The previously saved field can be loaded from the certain h5 file by providing its filename and time.

In [26]:
loaded_field = sim.llg.m_field.load_hdf5(filename, 4e-12)

Now, the elements from this array can be shown:

In [31]:
loaded_field.vector().array()

array([ 0.72294017,  0.42182628, -0.59223996, -0.38197242,  0.61394026,
        0.3532982 , -0.62971874, -0.3235068 ,  0.65529298,  0.29414287,
       -0.50209312, -0.35089212,  0.41826458,  0.51057019, -0.44469894,
       -0.49039961,  0.46868014,  0.47265406, -0.4900798 , -0.44979544,
        0.51702303,  0.6189359 , -0.65575968, -0.53306943,  0.46384288,
        0.51839873, -0.48643252, -0.50443341,  0.50438803,  0.49025671,
       -0.52642727, -0.47707367,  0.3306934 ,  0.31930661, -0.46549863,
       -0.51848402,  0.48158754,  0.50394088, -0.49599606, -0.49429373,
        0.50820115,  0.48544717, -0.51442969, -0.68388004,  0.67697627,
        0.51106036, -0.48083593, -0.50583814,  0.49261386,  0.50073673,
       -0.50152701, -0.4976837 ,  0.51169743,  0.49896541, -0.29210569,
       -0.29480439,  0.48701652,  0.50929208, -0.49284213, -0.50280036,
        0.49763892,  0.50259729, -0.4996328 , -0.50431937,  0.49238904,
        0.70153919, -0.68966667, -0.49634251,  0.49419845,  0.50